Мы будем работать с Postgres, так что надо проинсталлировать [библиотеку](https://pythonru.com/biblioteki/vvedenie-v-postgresql-s-python-psycopg2) для работы с ним.

In [36]:
!pip3 install mysql-connector-python

In [1]:
# Подключаем библиотеки.
import mysql.connector



Сперва надо создать соединение с сервером, не забыв указать где находится сервер, с какой базой данных мы собираемся работать, логин и пароль для пользователя.

In [3]:
con = mysql.connector.connect(host="localhost", port=33007, user="root", password="my-secret-pw")

In [4]:
cur = con.cursor()

In [12]:
cur.execute("CREATE SCHEMA IF NOT EXISTS nplus1")

In [13]:
cur.execute("""CREATE TABLE IF NOT EXISTS nplus1.articles (
  id_article INT NOT NULL AUTO_INCREMENT,
  article_text VARCHAR(8192) NULL,
  article_title VARCHAR(256) NULL,
  article_author VARCHAR(64) NULL,
  article_difficulty FLOAT NULL,
  PRIMARY KEY (`id_article`));""")

In [15]:
con = mysql.connector.connect(host="localhost", port=33007, database="nplus1", user="root", password="my-secret-pw")

Далее создаем курсор для отправки запросов и получения результатов. Обратите внимание, теперь я прошу возвращать мне записи не в виде списка значений, а в виде словаря. Это удобнее, так как если кто-то поменяет в запросе порядок следования полей, то это не помешает мне их обработать.

In [16]:
cur = con.cursor(dictionary=True)

Выполним несложный запрос при помощи функции execute - выбрать все поля и все записи из таблицы articles. 

In [17]:
cur.execute("SELECT * FROM articles")

Далее надо выбрать данные из курсора.

In [18]:
res = cur.fetchall()

In [19]:
res

[]

Итак, у нас получилось. Хотя ничего и не прочиталось, так как таблица пуста, но получилось.

Теперь посмотрим как можно добавлять записи при помощи INSERT INTO.

In [43]:
cur.execute("""INSERT INTO articles (article_text, article_title, article_author, article_difficulty) 
               VALUES ('bla-bla-bla3','bla333-bla-bla','John Doe',2.4), 
                      ('bla-bla-bla2','bla-bla-bla2','John Doe','5.4')""")

После изменений данных необходимо подтвердить при при помощи функции соединения `commit()` или откатить назад при помощи `rollback()`.

In [64]:
con.commit()

Теперь загрузим статьи из NPlus1 и положим их в базу.

In [20]:
import requests
from lxml import html

In [24]:
class NPlus1Article:
    def __init__(self):
        self.time = ""
        self.date = ""
        self.rubr = ""
        self.diff = ""
        self.author = ""
        self.head = ""
        self.text = ""
        
    def __repr__(self) -> str:
        return f"{self.head}, {self.author}, {self.date}, {self.rubr}, {self.text}"

def getArticleTextNPlus1(adr):
    page = requests.get(adr)
    #print(r.text)
    art=NPlus1Article()
    tree = html.fromstring(page.text)
    art.head = tree.xpath(".//h1")[0].text_content()
    dt = tree.xpath(".//time//span")
    art.date = dt[1].text_content()
    art.time = dt[0].text_content()
    art.rubr = tree.xpath(".//a[@data-rubric]")[0].text_content()
    art.diff = tree.xpath(".//span[@class='difficult-value']")[0].text_content()
    art.author = tree.xpath(".//meta[@name='mediator_author']")[0].get("content")
    art.text = '\n'.join([p.text_content() for p in tree.xpath(".//article//p")[3:-2]])
    return art

def getDayArticles(adr):
    r = requests.get(adr)
    titles = BeautifulSoup(r.text, "html5lib")("article")
    #print(titles)
    addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
    #print(addrs)
    articles = []
    for adr in addrs:
        print(adr)
        articles.append(getArticleTextNPlus1(adr))
    return articles

In [25]:
arts = getDayArticles("https://nplus1.ru/news/2021/12/17/")

https://nplus1.ru//news/2021/12/17/tardigrade-entanglement
https://nplus1.ru//news/2021/12/17/w-1243
https://nplus1.ru//news/2021/12/17/photo-radiofluorination
https://nplus1.ru//news/2021/12/17/warrior-burial
https://nplus1.ru//news/2021/12/17/a380
https://nplus1.ru//news/2021/12/17/rectum-ventilation
https://nplus1.ru//news/2021/12/17/ancient-diet
https://nplus1.ru//news/2021/12/17/rapiddragon
https://nplus1.ru//news/2021/12/17/mariner-water


Посмотрим что вообще загружается.

In [26]:
arts[0].head

'Физики встроили тихоходку в кубит'

SQL-запрос можно формировать динамически - это всего лишь строка.

In [27]:
req="""INSERT INTO articles (article_text, article_title, article_author, article_difficulty)
       VALUES ('{}','{}','{}',{})""".format(arts[0].text, arts[0].head, arts[0].author, arts[0].diff)
req

"INSERT INTO articles (article_text, article_title, article_author, article_difficulty)\n       VALUES ('\nK. S. Lee et al. / arXiv.org, 2021\nУченые из\xa0нескольких стран доложили о\xa0результатах эксперимента, в\xa0ходе которого они использовали тихоходку, чтобы изменить состояние трансмонного кубита, запутанного с\xa0другим кубитом в\xa0условиях очень низких температур и\xa0давлений. Тихоходка пережила этот эксперимент, обновив рекорд по\xa0экстремальности физических условий, которые она может выдержать. Препринт работы опубликован на\xa0сайте arxiv.org, а\xa0сама она пока не\xa0прошла рецензирование.\nКвантовая суперпозиция\xa0— это феномен, возникающий в\xa0квантовой механике, который не\xa0имеет аналогов в\xa0классической картине мира. Он\xa0заключается в\xa0том, что квантовые состояния объектов могут быть представлены в\xa0виде суммы состояний, соответствующих различным значениям какой-либо физической наблюдаемой. Если в\xa0системе присутствует несколько взаимодействующих объек

In [30]:
cur.execute(req)

In [31]:
con.commit()

Положим статьи с сайта в базу данных. Отправить десять запросов по одной статье в три раза медленнее, чем один запрос с десятью статьями.

In [32]:
%%time
for art in arts:
    req="INSERT INTO articles (article_text, article_title, article_author, article_difficulty) VALUES ('{}','{}','{}',{})".format(art.text.replace("'", "\\\'"), art.head, art.author, art.diff)
    cur.execute(req)
    con.commit()

CPU times: user 118 µs, sys: 4.07 ms, total: 4.19 ms
Wall time: 44.3 ms


In [33]:
%%time
data=[]
for art in arts:
    data.append("('{}','{}','{}',{})".format(art.text.replace("'", "\\\'"), art.head, art.author, art.diff))

req="INSERT INTO articles (article_text, article_title, article_author, article_difficulty) VALUES "
req+=",".join(data)    
cur.execute(req)
con.commit()

CPU times: user 1.53 ms, sys: 93 µs, total: 1.62 ms
Wall time: 12.8 ms


Удалим за собой все статьи

In [34]:
cur.execute("DELETE FROM articles")

In [35]:
con.commit()

In [34]:
con.rollback()

---

### Пару слов о безопасности

![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

Вот так гораздо лучше.

```Python
req = f"INSERT INTO articles (title, date, article, id_author, abstract) VALUES " \
      f"(%s, date(%s), %s, %d, %s)"
cur.execute(req, (head, art.date, text, id_author, abstract))
con.commit()
```

Заодно не надо будет заменять апострофы (и кое-что ещё).

~~~~sql
INSERT INTO lenta_ru.articles (title, date, article, id_author, abstract) VALUES
('Robert');
DROP TABLE Students; --', date('1111'), '22222', 333, '44444')
~~~~

Если в тексте какой-нибудь новости встретится апостроф, то запрос не выполнится, так как MySQL подумает, что в этом месте заканчивается строка. В связи с этим все апострофы надо заменить на \\'.

In [42]:
cur.execute("select * from articles WHERE title='O'Rly'")

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Rly'' at line 1

In [43]:
con.rollback()

In [46]:
cur.execute("select * from articles WHERE article_title='O\\'Rly'")
res = cur.fetchall()
res

[]